# 未完成

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from scipy.io import loadmat
%matplotlib inline 

In [2]:
#load the data 
X=loadmat('ex4data1.mat')['X']
X.shape

(5000, 400)

In [3]:
y=loadmat('ex4data1.mat')['y']
#y=y.ravel()  #将y转变为一维向量

In [4]:
# load 训练好的参数
theta1=loadmat('ex4weights.mat')['Theta1']
theta2=loadmat('ex4weights.mat')['Theta2']

In [5]:
#构建sigmoid
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [6]:
#对y进行‘one-hot' 处理
def expand(y):
    res = []
    for i in y:
        y_array = np.zeros(10)
        y_array[i - 1] = 1
        res.append(y_array)
    return np.array(res)

In [38]:
y = expand(y)
y

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [41]:
def feed_forward(theta, X):
    """apply to architecture 400+1 * 25+1 *10
    X: 5000 * 401
    """

    t1, t2 = deserialize(theta)  # t1: (25,401) t2: (10,26)
    m = X.shape[0]
    a1 = X  # 5000 * 401

    z2 = a1 @ t1.T  # 5000 * 25
    a2 = np.insert(sigmoid(z2), 0, np.ones(m), axis=1)  # 5000*26

    z3 = a2 @ t2.T  # 5000 * 10
    h = sigmoid(z3)  # 5000*10, this is h_theta(X)

    return a1, z2, a2, z3, h  # you need all those for backprop


In [39]:
def deserialize(seq):
#     """into ndarray of (25, 401), (10, 26)"""
    return seq[:25 * 401].reshape(25, 401), seq[25 * 401:].reshape(10, 26)

In [40]:
def serialize(a, b):
  
    return np.concatenate((np.ravel(a), np.ravel(b)))
# 序列化2矩阵
# 在这个nn架构中，我们有theta1（25,401），theta2（10,26），它们的梯度是delta1，delta2  

In [44]:
theta = serialize(theta1, theta2) # 扁平化参数，25*401+10*26=10285

In [45]:
feed_forward(theta,X)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 401 is different from 400)

In [10]:
# 定义损失函数
def cost(theta1,theta2, X, y):
#     """calculate cost
#     y: (m, k) ndarray
#     """
    m = X.shape[0]  # get the data size m

    _, _, _, _, h = forward_propagate(X,theta1,theta2)

    # np.multiply is pairwise operation
    pair_computation = -np.multiply(y, np.log(h)) - np.multiply((1 - y), np.log(1 - h))

    return pair_computation.sum() / m


In [11]:
cost(theta1,theta2,X,y)

1109.389991128097

In [12]:
# 正则化损失函数
def r_cost(theta1,theta2,X,y,l=1):
    m = X.shape[0]
    tmp = np.square(theta1[:,1:]).sum() + np.square(theta2[:,1:]).sum()
    return cost(theta1,theta2,X,y) + tmp * (l/(2*m))
    

In [13]:
r_cost(theta1,theta2,X,y)

1109.4861318220267

## 实现向后传播（Backpropagation)

In [29]:
# Sigmoid gradient
def sigmoidGradient(z):
    return np.multiply(sigmoid(z) ,(1-sigmoid(z)))

In [15]:
sigmoidGradient(X)

array([[0.25, 0.25, 0.25, ..., 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, ..., 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, ..., 0.25, 0.25, 0.25],
       ...,
       [0.25, 0.25, 0.25, ..., 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, ..., 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, ..., 0.25, 0.25, 0.25]])

In [16]:
# Random initialization
def random_init(size):
    return np.random.uniform(-0.12, 0.12, size)

In [17]:
random_init(5)

array([ 0.00119368,  0.10871885, -0.02534946,  0.02731261,  0.05427623])

In [18]:
#forward_propagate(theta1,theta2,X)

In [32]:
def gradient(theta1,theta2, X, y):
  
    m = X.shape[0]

    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    a1, z2, a2, z3, h = forward_propagate(X,theta1,theta2)

    for i in range(m):
        a1i = a1[i, :]  # (1, 401)
        z2i = z2[i, :]  # (1, 25)
        a2i = a2[i, :]  # (1, 26)

        hi = h[i, :]    # (1, 10)
        yi = y[i, :]    # (1, 10)

        d3i = hi - yi  # (1, 10)

        z2i = np.insert(z2i, 0, np.ones(1))  # make it (1, 26) to compute d2i
        d2i = np.multiply(theta2.T.dot(d3i), sigmoidGradient(z2i))  # (1, 26)

        # careful with np vector transpose
        delta2 += np.matrix(d3i).T @ np.matrix(a2i)  # (1, 10).T @ (1, 26) -> (10, 26)
        delta1 += np.matrix(d2i[1:]).T @ np.matrix(a1i)  # (1, 25).T @ (1, 401) -> (25, 401)

    delta1 = delta1 / m
    delta2 = delta2 / m

    return (delta1, delta2)

In [36]:
d1,d2=gradient(theta1,theta2,X,y)
d1.shape
d2.shape

(10, 26)